# **Grouped `bureau_balance`**

# Data Loading and Preprocessing

**Time:** 8.5 s for 27,299,925 entries.

In [1]:
from home_credit.tables import BureauBalance
from pepper.univar import print_value_counts_dict

data = BureauBalance.clean()
print_value_counts_dict(data, "STATUS")
display(data)
# ok data.info()

load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\bureau_balance.pqt
load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\bureau.pqt
load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\application_train.pqt
load C:/Users/franc/Projects/pepper_credit_scoring_tool\dataset\pqt\application_test.pqt
STATUS (7): {'C': 12006499, '0': 11799930, '1': 273013, '5': 59328, '2': 24091, '3': 9113, '4': 7767}


CLEAN_BUREAU_BALANCE         TARGET  SK_ID_CURR STATUS
SK_ID_BUREAU MONTHS_BALANCE                           
5001710      0                   -1      162368      C
             1                   -1      162368      C
             2                   -1      162368      C
             3                   -1      162368      C
             4                   -1      162368      C
...                             ...         ...    ...
6842888      57                   0      387020      0
             58                   0      387020      1
             59                   0      387020      0
             60                   0      387020      0
             61                   0      387020      0

[24179741 rows x 3 columns]

# Key Uniqueness

We verify that there cannot be multiple `SK_ID_CURR` for one `SK_ID_BUREAU`.

The issue is therefore multi-indexed only in appearance: the `SK_ID_CURR` key is sufficient to separate the groups.

Number of `SK_ID_BUREAU` for one `SK_ID_CURR` and vice versa :

In [6]:
from home_credit.merge import _get_unique_and_multi_index, curr_prev_uniqueness_report

# Get unique and multi-indexes for the specified table and columns
indexes = _get_unique_and_multi_index(data.reset_index(), "SK_ID_BUREAU", "SK_ID_CURR")

# Generate a report on the uniqueness of SK_ID_CURR and SK_ID_BUREAU
curr_prev_uniqueness_report(*indexes)

number of unique (curr, prev)              : 774 354
number of curr with more than 1 prev       : 756 714
number of curr with one prev               : 17 640
number of curr with more than 1 prev (in %): 97.7
number of prev with more than 1 curr       : 0
number of prev with one curr               : 774 354
number of prev with more than 1 curr (in %): 0.0


# Agrégation cf. **`old_kernel_v2`**

Le premier jet était inspiré du **`lightgbm_kernel`**, un kernel de référence disponible sur Kaggle.

Il s'agit d'une agrégation par prêt qui produit 774 354 échantillons de synthèse.

L'information est appauvrie, on obtient :
- les premier, dernier et nombre de mois de suivi (96 maximum).
- les fréquences sur la période de suivi des occurrences de chaque modalité de `STATUS`.

In [8]:
from home_credit.kernel import hot_encode_cats

encoded_data, cat_vars = hot_encode_cats(data.reset_index())
months_agg_rules = {"MONTHS_BALANCE": ["min", "max", "size"]}
cat_vars_agg_rules = {col: ["mean"] for col in cat_vars}
agg_rules = months_agg_rules | cat_vars_agg_rules
aggregated = encoded_data.groupby("SK_ID_BUREAU").agg(agg_rules)
display(aggregated)

MONTHS_BALANCE           STATUS_0  STATUS_1 STATUS_2 STATUS_3  \
                        min max size      mean      mean     mean     mean   
SK_ID_BUREAU                                                                 
5001710                   0  82   83  0.421687  0.000000      0.0      0.0   
5001711                   0   3    4  1.000000  0.000000      0.0      0.0   
5001712                   0  18   19  0.526316  0.000000      0.0      0.0   
5001713                   0  21   22  1.000000  0.000000      0.0      0.0   
5001714                   0  14   15  1.000000  0.000000      0.0      0.0   
...                     ...  ..  ...       ...       ...      ...      ...   
6842884                   0  47   48  0.208333  0.000000      0.0      0.0   
6842885                   0  23   24  0.500000  0.000000      0.0      0.0   
6842886                   0  32   33  0.242424  0.000000      0.0      0.0   
6842887                   0  36   37  0.162162  0.000000      0.0      0.0   
6842888                   0  61   62  0.145161  0.016129      0.0      0.0   

             STATUS_4 STATUS_5  STATUS_C  
                 mean     mean      mean  
SK_ID_BUREAU                              
5001710           0.0      0.0  0.578313  
5001711           0.0      0.0  0.000000  
5001712           0.0      0.0  0.473684  
5001713           0.0      0.0  0.000000  
5001714           0.0      0.0  0.000000  
...               ...      ...       ...  
6842884           0.0      0.0  0.791667  
6842885           0.0      0.5  0.000000  
6842886           0.0      0.0  0.757576  
6842887           0.0      0.0  0.837838  
6842888           0.0      0.0  0.838710  

[774354 rows x 10 columns]

# Agrégation du niveau de risque (`STATUS`)

## Rappel à propos de la variable catégorielle `STATUS`

* $21\,\%$ de NA cachés sous le code `X`.
* $50\,\%$ de prêt clôturés, code `C`, ce qui est une proportion importante et pose question.
* $27\,\%$ de cas sans problème particulier.
* $12\,\%$ distribués sur 5 classes de défauts de paiement :
    * $9\,\%$ en classe `1` - Défaut maximum atteint dans les 30 derniers jours
    * $1\,\%$ en classe `2` - Défaut maximum atteint le mois précédent
    * $\varepsilon$ en classe `3` - Défaut maximum atteint il y a deux mois
    * $\varepsilon$ en classe `4` - Défaut maximum atteint il y a trois mois
    * $2\,\%$ en classe `5` - Défaut maximum atteint il y a plus de 4 mois, ou bien a été vendu, ou radié en tant que dette irrécouvrable

On lit entre les lignes qu'en cas de défaut, soit le recouvrement intervient très rapidement, le mois suivant, et la situation se régularise, ou bien qu'elle ne se régularise pas (les $\varepsilon$ sur 3 et 4).

Il est peu probable qu'un client en situation 5 obtienne à nouveau un prêt.

## Conception d'un score de risque synthétique

Pour le risque synthétique faisant abstraction du longitudinal, dans la première version, nous avons utilisé la moyenne.

C'est discutable si l'on considère ces deux aspects :
1. **comportemental** : si le client a déjà eu des incidents par le passé, a-t-il cherché à les résoudre rapidement, ou s'est-il installé dans des DPD longs.
2. **circonstanciel** : s'il y a du DPD en cours, ou récent, la santé financière actuelle du demandeur est fragile.

L'idée est donc de prendre plutôt le DPD maximum rencontré, mais avec un estompement progressif avec le temps.

Notons $s(m)$ le `STATUS` du `MONTHS_BALANCE` $m$ avec $m=0, \ldots, 95$, 0 représentant l'actuel, et 95 le plus ancien, à 4 ans d'aujourd'hui.

Les valeurs `X` et `C` sont encodées en 0.

Le niveau de risque est (voir section sur le risque d'impayé) est $e^{s(m)}$. 

Si le client a plusieurs prêts simultanés $(s_i)$, le niveau de risque du mois $m$ est $\displaystyle \sum_i e^{s_i(m)}$ et l'indice $s(m)$ agrégé est donc $\displaystyle s(m) = \log \sum e^{s_i(m)}$.

Une façon de produire un risque synthétique avec abstraction longitudinale est de diminuer l'impact de chaque terme $e^{s_i(m)}$ de la suite longitudinale par un facteur fonction de l'ancienneté.

On permet de moduler la force de ce facteur à l'aide d'un paramètre $\alpha$ par défaut à $1$ : $\displaystyle f_i(m) = \frac{e^{s_i(m)}}{1+m\alpha}$.

Ainsi, pour un défaut de paiement en cours $f_i(0) = e^{s_i(m)}$ et pour un défaut ancien, par ex. 4 ans, $\displaystyle f_i(95) = \frac{e^{s_i(95)}}{1+95\alpha}$.

La valeur moyenne du risque actualisé pour un prêt est alors :

$$\displaystyle S_i = \log \left(\frac{1}{M} \sum \frac{e^{s_i(m)}}{1+m\alpha}\right)$$

Où $M$ est le nombre de mois de suivi du client, pour un prêt particulier.

Celui du risque tous prêts confondus est alors :

$$\displaystyle S = \log \left(\frac{1}{M} \sum \frac{e^{s(m)}}{1+m\alpha}\right)$$

Où $M$ est le nombre de mois de suivi du client, tous prêts confondus.

## Agrégation par (prêt, mois) : c'est la base de fait

Simple encodage numérique de la colonne `STATUS`.

**Temps :** 3 s.

In [4]:
from home_credit.groupby import get_bureau_loan_status_by_month

status = get_bureau_loan_status_by_month(data)
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_MONTH  STATUS
SK_ID_BUREAU MONTHS_BALANCE        
5001710      0                    0
             1                    0
             2                    0
             3                    0
             4                    0
...                             ...
6842888      57                   0
             58                   1
             59                   0
             60                   0
             61                   0

[24179741 rows x 1 columns]

## Agrégation par (client, mois)

Ici, une véritable opération d'agrégation est effectuée.

On passe de 24 179 741 à 7 365 660 enregistrements.

**Temps :** 12 s.

In [7]:
from home_credit.groupby import get_bureau_loan_status_by_client_and_month

status = get_bureau_loan_status_by_client_and_month(data)
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH  STATUS
SK_ID_CURR MONTHS_BALANCE                     
100001     0                              0.22
           1                              0.00
           2                              0.00
           3                              0.00
           4                              0.00
...                                        ...
456255     72                             0.00
           73                             0.00
           74                             0.00
           75                             0.00
           76                             0.00

[7365660 rows x 1 columns]

## Agrégation par prêt

Ici, on effectue la synthèse longitudinale, en conservant un indice qui représente le risque d'impayé, cf. sa dynamique sur l'ensemble des (au plus 96) mois précédents suivis.

On part de la base de fait, et on effectue l'agrégation $\displaystyle S_i = \log \left(\frac{1}{M} \sum \frac{e^{s_i(m)}}{1+m\alpha}\right)$.

Cela nous donne 774 354 scores de risque par prêt.

**Temps :** 7 s.

In [9]:
from home_credit.groupby import get_bureau_loan_status

status = get_bureau_loan_status(data)
display(status)
# ok status.info()

BUREAU_LOAN_STATUS,STATUS
SK_ID_BUREAU,
5001710,-2.81
5001711,-0.65
5001712,-1.68
5001713,-1.79
5001714,-1.51
...,...
6842884,-2.38
6842885,2.96
6842886,-2.09


## Agrégation par client

Là, c'est moins direct que le précédent.

Il faut partir de la pré-agrégation par (client, mois) qui effectue la synthèse mensuelle du risque tous prêts confondus.

On obtient 134 542 enregistrements.

**Temps :** 13 s.

In [11]:
from home_credit.groupby import get_bureau_loan_status_by_client

status = get_bureau_loan_status_by_client(data)
display(status)
# ok status.info()

BUREAU_LOAN_STATUS_BY_CLIENT,STATUS
SK_ID_CURR,
100001,-2.44
100002,-2.33
100005,-1.41
100010,-3.10
100013,-2.66
...,...
456247,-2.80
456250,-2.09
456253,-2.04


# Agrégation du nombre de prêts actifs

On commence par produire cette information au niveau de la table de base, retraitée pour imputer les NA (avec une stratégie de fill forward). Un prêt est actif si `STATUS != C`. Les `STATUS = X`, sont des NA auquel l'imputation a donné une valeur valide.

Puis on forme le nombre des prêts actifs par mois pour chaque client.

Enfin, on réalise l'abstraction longitudinale, pour chaque prêt, puis pour chaque client. Comme pour le niveau de risque, nous utilisons une moyenne amortie pour que la situation la plus récente soit mieux représentée que la plus ancienne.

## Activité mensuelle des prêts

In [2]:
from home_credit.groupby import get_bureau_loan_activity_by_month

activity = get_bureau_loan_activity_by_month(data)
display(activity)
activity.info()

BUREAU_LOAN_ACTIVITY_BY_MONTH  ACTIVE
SK_ID_BUREAU MONTHS_BALANCE          
5001710      0                      0
             1                      0
             2                      0
             3                      0
             4                      0
...                               ...
6842888      57                     1
             58                     1
             59                     1
             60                     1
             61                     1

[24179741 rows x 1 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24179741 entries, (5001710, 0) to (6842888, 61)
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   ACTIVE  uint8
dtypes: uint8(1)
memory usage: 160.4 MB


## Nombre mensuel de prêts actifs par client

In [10]:
from home_credit.groupby import get_bureau_loan_activity_by_client_and_month

activity = get_bureau_loan_activity_by_client_and_month(data)
display(activity)
# ok activity.info()

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH  ACTIVE
SK_ID_CURR MONTHS_BALANCE                       
100001     0                                 3.0
           1                                 3.0
           2                                 2.0
           3                                 2.0
           4                                 2.0
...                                          ...
456255     72                                1.0
           73                                1.0
           74                                1.0
           75                                2.0
           76                                2.0

[7365660 rows x 1 columns]

In [9]:
from home_credit.groupby import get_bureau_mean_loan_activity_by_client

activity = get_bureau_mean_loan_activity_by_client(data)
display(activity)
# ok activity.info()

BUREAU_MEAN_LOAN_ACTIVITY_BY_CLIENT,ACTIVE
SK_ID_CURR,
100001,0.172164
100002,0.105258
100005,0.385651
100010,0.005784
100013,0.091223
...,...
456247,0.137085
456250,0.244074
456253,0.168209


## Activité mensuelle moyenne amortie par prêt

In [8]:
from home_credit.groupby import get_bureau_mean_loan_activity

activity = get_bureau_mean_loan_activity(data)
display(activity)
# ok activity.info()

BUREAU_MEAN_LOAN_ACTIVITY,ACTIVE
SK_ID_BUREAU,
5001710,0.006545
5001711,0.520833
5001712,0.037830
5001713,0.167764
5001714,0.221215
...,...
6842884,0.004810
6842885,0.157332
6842886,0.008268


## Activité mensuelle moyenne amortie par client

In [7]:
from home_credit.groupby import get_bureau_mean_loan_activity_by_client

activity = get_bureau_mean_loan_activity_by_client(data)
display(activity)
# ok activity.info()

BUREAU_MEAN_LOAN_ACTIVITY_BY_CLIENT,ACTIVE
SK_ID_CURR,
100001,0.172164
100002,0.105258
100005,0.385651
100010,0.005784
100013,0.091223
...,...
456247,0.137085
456250,0.244074
456253,0.168209


# Profils denses de variations à l'aide de l'encodage RLE

Il se peut que nous ayons besoin des informations longitudinales, soit brutes, soit agrégées (état, dénombrement, montant). Cela pourra servir d'abord au niveau des fusions avec les tables **`bureau`** et **`application`** si l'on choisit d'essayer une version à 96 dimensions d'une caractéristique ou d'une autre dont l'importance serait absolument déterminante (il faudra alors introduire une étape de réduction de dimensionnalité). Cela pourra également servir pour des agrégations transversales, pour un demandeur donné, entre les informations provenant de **`bureau`** et d'autres, de même nature, provenant de **`previous_application`**

Un pivotement entraînement l'apparition d'une quantité importante de cellules vides et nous forcerait à passer une gestion de matrices creuses, ce qui ajouterait une couche de complexité au code.

Pour pouvoir sauvegarder ces informations de manière dense, nous avons choisi d'utiliser la technique classique de compression RLE. Elle nous permet de condenser la suite de valeurs mensuelles d'un caractéristique, qui souvent reste constante sur l'ensemble de la période ou sur de longues sous-périodes.

Une relation de passage inverse nous permet de développer un vecteur de séquences codées RLE en un dataframe multi-dimensionnel qui serait le résultat d'un pivotement.

Il important de souligner que de nombreuses opérations (dériver un indicateur, union, intersection, etc) de suites peuvent être effectuée entre codes RLE sans repasser par une version développée.

Dans le cadre de la table **`bureau_balance`** chaque information que nous avons dégagée au grain mensuel peut être codifiée.

## Période de suivi

La période de suivi d'un prêt peut être la totalité des 96 mois, soit une sous-période, voire des sous-périodes fragmentées.

Notre première fonction, basée sur la fonction `jumps_rle` permet de codifier en RLE les sauts entre mois consécutifs de suivi.

### Période de suivi par prêt

On produit la table indexée par `SK_ID_BUREAU` des périodes de suivi des prêts, avec le premier, le dernier et le nombre de mois de suivi, et la représentation RLE des sous-périodes (dans la plupart des cas, une seule).

**Time :** 1 m.

In [4]:
from home_credit.groupby import get_rle_bureau_loan_tracking_period

tracking = get_rle_bureau_loan_tracking_period(data)
display(tracking)

CLEAN_BUREAU_BALANCE MONTHS_BALANCE                      
                                min max count   jumps_rle
SK_ID_BUREAU                                             
5001710                           0  82    83  ((1, 83),)
5001711                           0   3     4   ((1, 4),)
5001712                           0  18    19  ((1, 19),)
5001713                           0  21    22  ((1, 22),)
5001714                           0  14    15  ((1, 15),)
...                             ...  ..   ...         ...
6842884                           0  47    48  ((1, 48),)
6842885                           0  23    24  ((1, 24),)
6842886                           0  32    33  ((1, 33),)
6842887                           0  36    37  ((1, 37),)
6842888                           0  61    62  ((1, 62),)

[774354 rows x 4 columns]

### Période de suivi par client

On produit la table indexée par `SK_ID_CURR` des périodes de suivi, tous prêts confondus, avec le premier, le dernier et le nombre de mois de suivi, et la représentation RLE des sous-périodes (dans la plupart des cas, une seule).

Exemple pour aider à l'interprétation des codes RLE :

```
100010	2	90	72	((3, 1), (1, 35), (18, 1), (1, 35))
```

Indique une période courant les mois 2 à 90, mais avec 72 mois effectivement suivis, avec une première période de 36 mois, un _gap_ de de 18 mois, puis une nouvelle période de suivi de 36 mois.

Application : sur 134 542 suivis, 113 748 le sont sans interruption jusqu'à aujourd'hui.

**Time :** 22 s.

In [7]:
from home_credit.groupby import get_rle_bureau_loan_tracking_period_by_client

tracking = get_rle_bureau_loan_tracking_period_by_client(data)
display(tracking)

CLEAN_BUREAU_BALANCE MONTHS_BALANCE            \
                                min max count   
SK_ID_CURR                                      
100001                            0  51    52   
100002                            0  47    48   
100005                            0  12    13   
100010                            2  90    72   
100013                            0  68    69   
...                             ...  ..   ...   
456247                            0  81    82   
456250                            0  32    33   
456253                            0  30    31   
456254                            0  36    37   
456255                            0  76    77   

CLEAN_BUREAU_BALANCE                                       
                                                jumps_rle  
SK_ID_CURR                                                 
100001                                         ((1, 52),)  
100002                                         ((1, 48),)  
100005                                         ((1, 13),)  
100010                ((3, 1), (1, 35), (18, 1), (1, 35))  
100013                                         ((1, 69),)  
...                                                   ...  
456247                                         ((1, 82),)  
456250                                         ((1, 33),)  
456253                                         ((1, 31),)  
456254                                         ((1, 37),)  
456255                                         ((1, 77),)  

[134542 rows x 4 columns]

Exemple d'utilisation : nombre de périodes de suivi continues jusqu'à aujourd'hui :

In [6]:
rle_month_jumps = tracking[("MONTHS_BALANCE", "jumps_rle")]
is_continuous_tracking = rle_month_jumps.apply(lambda x: len(x) == 1)

print(f"# continuous tracking periods : {sum(is_continuous_tracking)}")

# continuous tracking periods : 113748


## Variations longitudinales (mensuelles)

Pour être interprétées, elles doivent toujours être associées à une période de suivi support.

### Variation d'activité par prêt

**Temps :** 4 m 15 s.

In [3]:
from home_credit.groupby import get_rle_bureau_loan_feature_variations
from home_credit.groupby import get_bureau_loan_activity_by_month

activity = get_bureau_loan_activity_by_month(data)
activity_variation = get_rle_bureau_loan_feature_variations(activity, "ACTIVE")
display(activity_variation)

BUREAU_LOAN_ACTIVITY_BY_MONTH MONTHS_BALANCE                        \
                                         min max count   jumps_rle   
SK_ID_BUREAU                                                         
5001710                                    0  82    83  ((1, 83),)   
5001711                                    0   3     4   ((1, 4),)   
5001712                                    0  18    19  ((1, 19),)   
5001713                                    0  21    22  ((1, 22),)   
5001714                                    0  14    15  ((1, 15),)   
...                                      ...  ..   ...         ...   
6842884                                    0  47    48  ((1, 48),)   
6842885                                    0  23    24  ((1, 24),)   
6842886                                    0  32    33  ((1, 33),)   
6842887                                    0  36    37  ((1, 37),)   
6842888                                    0  61    62  ((1, 62),)   

BUREAU_LOAN_ACTIVITY_BY_MONTH               ACTIVE  
                              series_rle_reduction  
SK_ID_BUREAU                                        
5001710                         ((0, 48), (1, 35))  
5001711                                  ((1, 4),)  
5001712                          ((0, 9), (1, 10))  
5001713                                 ((1, 22),)  
5001714                                 ((1, 15),)  
...                                            ...  
6842884                         ((0, 38), (1, 10))  
6842885                                 ((1, 24),)  
6842886                          ((0, 25), (1, 8))  
6842887                          ((0, 31), (1, 6))  
6842888                         ((0, 52), (1, 10))  

[774354 rows x 5 columns]

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 774354 entries, 5001710 to 6842888
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   (MONTHS_BALANCE, min)           774354 non-null  uint8 
 1   (MONTHS_BALANCE, max)           774354 non-null  uint8 
 2   (MONTHS_BALANCE, count)         774354 non-null  uint8 
 3   (MONTHS_BALANCE, jumps_rle)     774354 non-null  object
 4   (ACTIVE, series_rle_reduction)  774354 non-null  object
dtypes: object(2), uint8(3)
memory usage: 19.9+ MB


### Variation d'activité par client

**Temps :** 45 s.

In [4]:
from home_credit.groupby import get_rle_bureau_loan_feature_by_client_variations
from home_credit.groupby import get_bureau_loan_activity_by_client_and_month

activity = get_bureau_loan_activity_by_client_and_month(data)
activity_variation = get_rle_bureau_loan_feature_by_client_variations(activity, "ACTIVE")
display(activity_variation)
activity_variation.info()

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH MONTHS_BALANCE            \
                                                    min max count   
SK_ID_CURR                                                          
100001                                                0  51    52   
100002                                                0  47    48   
100005                                                0  12    13   
100010                                                2  90    72   
100013                                                0  68    69   
...                                                 ...  ..   ...   
456247                                                0  81    82   
456250                                                0  32    33   
456253                                                0  30    31   
456254                                                0  36    37   
456255                                                0  76    77   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                       \
                                                                    jumps_rle   
SK_ID_CURR                                                                      
100001                                                             ((1, 52),)   
100002                                                             ((1, 48),)   
100005                                                             ((1, 13),)   
100010                                    ((3, 1), (1, 35), (18, 1), (1, 35))   
100013                                                             ((1, 69),)   
...                                                                       ...   
456247                                                             ((1, 82),)   
456250                                                             ((1, 33),)   
456253                                                             ((1, 31),)   
456254                                                             ((1, 37),)   
456255                                                             ((1, 77),)   

BUREAU_LOAN_ACTIVITY_BY_CLIENT_AND_MONTH                                             ACTIVE  
                                                                       series_rle_reduction  
SK_ID_CURR                                                                                   
100001                                    ((3, 2), (2, 9), (1, 10), (2, 8), (1, 1), (0, ...  
100002                                    ((1, 4), (0, 9), (1, 3), (0, 1), (1, 1), (2, 3...  
100005                                                                    ((2, 3), (1, 10))  
100010                                                 ((0, 26), (1, 10), (0, 26), (1, 10))  
100013                                    ((1, 18), (2, 22), (1, 1), (2, 3), (3, 11), (2...  
...                                                                                     ...  
456247                                    ((2, 1), (3, 2), (4, 7), (3, 1), (2, 1), (1, 1...  
456250                                                    ((2, 25), (3, 1), (2, 2), (1, 5))  
456253                                                            ((1, 19), (4, 5), (3, 7))  
456254                                                                    ((0, 29), (1, 8))  
456255                                    ((0, 3), (2, 4), (3, 3), (4, 2), (3, 1), (2, 2...  

[134542 rows x 5 columns]

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 134542 entries, 100001 to 456255
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   (MONTHS_BALANCE, min)           134542 non-null  uint8 
 1   (MONTHS_BALANCE, max)           134542 non-null  uint8 
 2   (MONTHS_BALANCE, count)         134542 non-null  uint8 
 3   (MONTHS_BALANCE, jumps_rle)     134542 non-null  object
 4   (ACTIVE, series_rle_reduction)  134542 non-null  object
dtypes: object(2), uint8(3)
memory usage: 3.5+ MB


### Variation du niveau de risque par prêt

**Temps :** 4 m.

In [7]:
from home_credit.groupby import get_rle_bureau_loan_feature_variations
from home_credit.groupby import get_bureau_loan_status_by_month

status = get_bureau_loan_status_by_month(data)
status_variation = get_rle_bureau_loan_feature_variations(status, "STATUS")
display(status_variation)

BUREAU_LOAN_STATUS_BY_MONTH MONTHS_BALANCE                        \
                                       min max count   jumps_rle   
SK_ID_BUREAU                                                       
5001710                                  0  82    83  ((1, 83),)   
5001711                                  0   3     4   ((1, 4),)   
5001712                                  0  18    19  ((1, 19),)   
5001713                                  0  21    22  ((1, 22),)   
5001714                                  0  14    15  ((1, 15),)   
...                                    ...  ..   ...         ...   
6842884                                  0  47    48  ((1, 48),)   
6842885                                  0  23    24  ((1, 24),)   
6842886                                  0  32    33  ((1, 33),)   
6842887                                  0  36    37  ((1, 37),)   
6842888                                  0  61    62  ((1, 62),)   

BUREAU_LOAN_STATUS_BY_MONTH                     STATUS  
                                  series_rle_reduction  
SK_ID_BUREAU                                            
5001710                                     ((0, 83),)  
5001711                                      ((0, 4),)  
5001712                                     ((0, 19),)  
5001713                                     ((0, 22),)  
5001714                                     ((0, 15),)  
...                                                ...  
6842884                                     ((0, 48),)  
6842885                             ((5, 12), (0, 12))  
6842886                                     ((0, 33),)  
6842887                                     ((0, 37),)  
6842888                      ((0, 58), (1, 1), (0, 3))  

[774354 rows x 5 columns]

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 774354 entries, 5001710 to 6842888
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   (MONTHS_BALANCE, min)           774354 non-null  uint8 
 1   (MONTHS_BALANCE, max)           774354 non-null  uint8 
 2   (MONTHS_BALANCE, count)         774354 non-null  uint8 
 3   (MONTHS_BALANCE, jumps_rle)     774354 non-null  object
 4   (STATUS, series_rle_reduction)  774354 non-null  object
dtypes: object(2), uint8(3)
memory usage: 19.9+ MB


### Variation du niveau de risque par client

**TODO** : régler les problèmes d'arrondi dans le RLE, certainement liés à mon choix de réduction de la taille d'encodage : correction dans RLE réduction, avec l'ajout d'un paramètre de précision, qui est d'autant plus important qu'il permet de mieux synthétiser à mesure que la précision diminue.

Après analyse : c'est le `groupby` et la gestion des types non `float64` par pandas qui est en cause: les valeurs de mes groupes en `float32` ont changé à la sortie et sont des valeurs dégénérées (`0.22` est devenu `0.2199999988079071` alors que le groupement devrait se contenter de collecter les valeurs sans les modifier). La raison est la suivante : le groupby force le passage par np.float64 : en effet,`np.float4(0.22) = 0.2199999988079071` 

Les stratégies pour contourner :
1. ajouter un paramètre optionnel `decimals` à mon `series_rle_reduction` pour qu'il retraite via un `np.round` mais valeurs dégénérées par le `groupby` : c'est également une extension fonctionnelle, puisque permet de regrouper des termes proches.
2. moins intrusif et coûteux, simplement corriger le transtypage non sollicité imposé par le `groupby`, avec le passage du `dtype` d'origine à `series_rle_reduction`.
3. Avec un temps imparti réduit, rester sur du `float64` dans les tables de niveaux 2 et 3, et ne downcaster que dans le merge final.

In [9]:
from home_credit.groupby import get_rle_bureau_loan_feature_by_client_variations
from home_credit.groupby import get_bureau_loan_status_by_client_and_month

status = get_bureau_loan_status_by_client_and_month(data)
status_variation = get_rle_bureau_loan_feature_by_client_variations(status, "STATUS")
display(status_variation)

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH MONTHS_BALANCE            \
                                                  min max count   
SK_ID_CURR                                                        
100001                                              0  51    52   
100002                                              0  47    48   
100005                                              0  12    13   
100010                                              2  90    72   
100013                                              0  68    69   
...                                               ...  ..   ...   
456247                                              0  81    82   
456250                                              0  32    33   
456253                                              0  30    31   
456254                                              0  36    37   
456255                                              0  76    77   

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH                                       \
                                                                  jumps_rle   
SK_ID_CURR                                                                    
100001                                                           ((1, 52),)   
100002                                                           ((1, 48),)   
100005                                                           ((1, 13),)   
100010                                  ((3, 1), (1, 35), (18, 1), (1, 35))   
100013                                                           ((1, 69),)   
...                                                                     ...   
456247                                                           ((1, 82),)   
456250                                                           ((1, 33),)   
456253                                                           ((1, 31),)   
456254                                                           ((1, 37),)   
456255                                                           ((1, 77),)   

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH                                             STATUS  
                                                                     series_rle_reduction  
SK_ID_CURR                                                                                 
100001                                               ((0.2199999988079071, 1), (0.0, 51))  
100002                                  ((0.0, 19), (0.36000001430511475, 1), (0.62000...  
100005                                                                       ((0.0, 13),)  
100010                                                                       ((0.0, 72),)  
100013                                  ((0.0, 18), (0.36000001430511475, 1), (0.0, 4)...  
...                                                                                   ...  
456247                                                                       ((0.0, 82),)  
456250                                                                       ((0.0, 33),)  
456253                                                                       ((0.0, 31),)  
456254                                                                       ((0.0, 37),)  
456255                                                                       ((0.0, 77),)  

[134542 rows x 5 columns]

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 134542 entries, 100001 to 456255
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   (MONTHS_BALANCE, min)           134542 non-null  uint8 
 1   (MONTHS_BALANCE, max)           134542 non-null  uint8 
 2   (MONTHS_BALANCE, count)         134542 non-null  uint8 
 3   (MONTHS_BALANCE, jumps_rle)     134542 non-null  object
 4   (STATUS, series_rle_reduction)  134542 non-null  object
dtypes: object(2), uint8(3)
memory usage: 3.5+ MB


## Annexe (à déplacer) : Expansion RLE

D'un codage RLE au dataframe.

In [12]:
display(status)

BUREAU_LOAN_STATUS_BY_CLIENT_AND_MONTH  STATUS
SK_ID_CURR MONTHS_BALANCE                     
100001     0                              0.22
           1                              0.00
           2                              0.00
           3                              0.00
           4                              0.00
...                                        ...
456255     72                             0.00
           73                             0.00
           74                             0.00
           75                             0.00
           76                             0.00

[7365660 rows x 1 columns]

In [35]:
import numpy as np
x = status.loc[100001]["STATUS"].iloc[0]
display(x)
print(type(x))
print(np.float64(x))
print(np.float32(np.float64(x)))

0.22

<class 'numpy.float32'>
0.2199999988079071
0.22


In [38]:
s = status.loc[100001]["STATUS"]

display(s.values)

array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ], dtype=float32)

In [41]:
from pepper.feat_eng import series_rle_reduction
r = series_rle_reduction(s)
print(r)
print(type(r))

((0.2199999988079071, 1), (0.0, 51))
<class 'tuple'>


In [49]:
import numpy as np
r = ((0, 1), (1, 2), (2, 3))
display(np.repeat(*(np.array(r).T)))

array([0, 1, 1, 2, 2, 2])

In [67]:
r = series_rle_reduction(s)
a = np.array(r)
display(a)
values, counts = np.split(np.array(r), 2, axis=1)
counts = counts.astype(int)
print(values[:, 0])
print(counts[:, 0])
display(np.repeat(values[:, 0], counts[:, 0]))

array([[ 0.22,  1.  ],
       [ 0.  , 51.  ]])

[0.22 0.  ]
[ 1 51]


array([0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [74]:
from typing import Union
import numpy as np



def rle_expand_expr(
    rle_expr: Union[list, tuple, np.ndarray]
) -> np.ndarray:
    """
    Expand a series from its RLE (Run-Length Encoding) representation,
    which is given as a sequence of value-count pairs.

    Parameters
    ----------
    rle_expr : Union[list, tuple, np.ndarray]
        The RLE representation of the series,
        typically obtained from series_rle_reduction.

    Returns
    -------
    np.ndarray
        The original series as a NumPy array.
        
    Example
    -------
    >>> rle_expand(((0, 1), (1, 2), (2, 3)))
    array([0, 1, 1, 2, 2, 2])
    >>> rle_expand([[1, 1], [0, 2], [.5, 1]])
    array([1. , 0. , 0. , 0.5])
    >>> rle_series.apply(rle_expand_expr)
    
    Notes
    -----
    This function is essentially a wrapper for the Numpy `repeat` function.
    
    Raises
    ------
    ValueError
        If the input format is not valid, i.e.,
        if it doesn't consist of value-count pairs.
    """
    # Convert input to a NumPy array if it's not already
    if not isinstance(rle_expr, np.ndarray):
        rle = np.array(rle_expr)

    if rle.size == 0:
        return np.array([])

    # Extract values and counts
    if rle.shape[1] == 2:
        values, counts = rle[:, 0], rle[:, 1].astype(int)
    else:
        raise ValueError("Invalid input format. Input must consist of value-count pairs.")

    # Expand the series
    expanded_series = np.repeat(values, counts)

    # Determine the dtype based on the type of the first element
    dtype = type(values[0])

    return expanded_series.astype(dtype)


display(rle_expand_expr(((0, 1), (1, 2), (2, 3))))
display(rle_expand_expr([[1, 1], [0, 2], [.5, 1]]))

array([0, 1, 1, 2, 2, 2])

array([1. , 0. , 0. , 0.5])

In [76]:
# display(activity_variation)
rle_series = activity_variation[("ACTIVE", "series_rle_reduction")]
display(rle_series)

SK_ID_CURR
100001    ((3, 2), (2, 9), (1, 10), (2, 8), (1, 1), (0, ...
100002    ((1, 4), (0, 9), (1, 3), (0, 1), (1, 1), (2, 3...
100005                                    ((2, 3), (1, 10))
100010                 ((0, 26), (1, 10), (0, 26), (1, 10))
100013    ((1, 18), (2, 22), (1, 1), (2, 3), (3, 11), (2...
                                ...                        
456247    ((2, 1), (3, 2), (4, 7), (3, 1), (2, 1), (1, 1...
456250                    ((2, 25), (3, 1), (2, 2), (1, 5))
456253                            ((1, 19), (4, 5), (3, 7))
456254                                    ((0, 29), (1, 8))
456255    ((0, 3), (2, 4), (3, 3), (4, 2), (3, 1), (2, 2...
Name: (ACTIVE, series_rle_reduction), Length: 134542, dtype: object

In [77]:
display(rle_series.apply(rle_expand_expr))

SK_ID_CURR
100001    [3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, ...
100002    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...
100005              [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
100010    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
100013    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                                ...                        
456247    [2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 0, 0, ...
456250    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
456253    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
456254    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
456255    [0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 4, 4, 3, 2, 2, ...
Name: (ACTIVE, series_rle_reduction), Length: 134542, dtype: object

In [78]:
np.repeat([1, np.nan, 2], [1, 2, 3])

array([ 1., nan, nan,  2.,  2.,  2.])

Il est facile d'étendre une series de rle_expr comme cela : rle_series.apply(rle_expand_expr)

Mais j'ai fait quelque chose d'assez fin : mes expressions rle résument les cellules non NA d'un tableau à 96 dimensions.

Avec une seconde fonction rle_jumps, je code le support, c'est à dire les indices des positions successives non NA, en partant (prepend) de -1 : par exemple (1, 3)(3, 1)(1, 3) donnera les indices suivants : 0, 1, 2, 5, 6, 7, 8. Mon but est de faire une fonction qui prend en argument deux series, celle qui code les supports et celle qui code les valeurs, et de produire en sortie un dataframe.

La première étape me semble d'être d'identifier la dimension du dataframe de sortie, qui est finalement donné par le support le plus long. Donc la première première étape est d'extraire cette information d'un codage rle de support, ce qui n'est pas très compliqué.

Ensuite, il faut trouver la succession d'opérations la plus efficace pour réaliser l'expansion.

Pour tirer parti de la fonction précédente, basée sur repeat, donc efficace, je pense que la meilleure stratégie est de modifier les rle_expr en fonction des rle_jumps, pour y insérer des 'trous', c'est à dire des répétitions de np.nan là où le support l'indique. Enfin, on utilie notre fonction précédente avec un apply sur les rle_expr ainsi transformée. 

In [79]:
jumps_rle_series = activity_variation[("MONTHS_BALANCE", "jumps_rle")]
display(jumps_rle_series)

SK_ID_CURR
100001                             ((1, 52),)
100002                             ((1, 48),)
100005                             ((1, 13),)
100010    ((3, 1), (1, 35), (18, 1), (1, 35))
100013                             ((1, 69),)
                         ...                 
456247                             ((1, 82),)
456250                             ((1, 33),)
456253                             ((1, 31),)
456254                             ((1, 37),)
456255                             ((1, 77),)
Name: (MONTHS_BALANCE, jumps_rle), Length: 134542, dtype: object

In [85]:
import numpy as np
def support_max_indice(
    rle_support_expr: Union[list, tuple, np.ndarray]
) -> int:
    # Convert input to a NumPy array if it's not already
    if not isinstance(rle_support_expr, np.ndarray):
        rle_support = np.array(rle_support_expr)

    if rle_support.size == 0:
        return -1

    # Extract values and counts
    if rle_support.shape[1] == 2:
        jumps, counts = rle_support[:, 0], rle_support[:, 1]
    else:
        raise ValueError("Invalid input format. Input must consist of jump-count pairs.")
    
    return -1 + np.sum(jumps * counts)

In [88]:
print(support_max_indice(((1, 52),)))
print(support_max_indice(((3, 1), (1, 35), (18, 1), (1, 35))))
max_indices = jumps_rle_series.apply(support_max_indice)
output_dim = max_indices.max() + 1
print(f"max_max_indice + 1 = dim: {output_dim}")
display(max_indices)

51
90
max_max_indice + 1 = dim: 97


SK_ID_CURR
100001    51
100002    47
100005    12
100010    90
100013    68
          ..
456247    81
456250    32
456253    30
456254    36
456255    76
Name: (MONTHS_BALANCE, jumps_rle), Length: 134542, dtype: int64

Idée :

Avec un support $(1, N)$, il n'y a rien à faire, séquence continue, pas de trous. L'opération de transformation de la rle_values est l'identité (c'est le cas le plus fréquent en la circonstance de nos données Home Credit).

Avec un support fragmenté, de la forme $(1, i)(k, j)(1, l)$ : il y a un trou de taille $k-1$ après la séquence des $i$ premières valeurs, suivi des $l+1$ valeurs suivantes.

Avec $k_j > 1$,

Avec un support $(1, i_1)(k_1, j_1)(1, i_2)(k_2, j_2)\ldots(1, i_n)(k_n, j_n)(1, i_{n+1})$, il y a $n$ _trous_ de tailles $k_1-1, \dots, k_n-1$, précédés de $i_1, i_2+1, i_3+1, \ldots, i_n+1$ valeurs.

Attention au cas où cela démarre par un trou !

Cela revient donc à insérer $n$ `(np.nan, k_j-1)` dans l'expression RLE des valeurs.

Là il faut réfléchir un peu pour le faire en Numpy, mais en gros, les distances d'insertion sont données par les $i_1, i_2+1, i_3+1, \ldots, i_n+1$. Par exemple, pour la première insertion, on identifie dans la séquence de valeurs, l'élément répété, tel que $i_1$ soit supérieur strictement à la somme des répétitions des éléments précédents, mais inférieure ou égale à cette somme augmentée du nombre de répétitions de cet élément.

Là, il faut éventuellement fragmenter l'élément en deux parties, celle qui précède le trou et celle qui le suit.

On procède à l'insertion.

La suite et une répétition de ce procédé.

In [90]:
import numpy as np

def rle_expr_to_numpy(
    rle_expr: Union[list, tuple, np.ndarray]
) -> np.array:
    # Convert inputs to NumPy arrays if they are not already
    return rle_expr if isinstance(rle_expr, np.ndarray) else np.array(rle_expr)

def rle_support_size(rle: np.array):
    return np.sum(rle[:, 1])


In [106]:
import numpy as np
support = ((1, 3), (3, 1), (1, 3))
values = ((1, 1), (2, 3), (3, 1))
def n_holes(rle_support: np.array):
    return np.sum(rle_support[:, 0] > 1)

rle_support = rle_expr_to_numpy(support)
rle_values = rle_expr_to_numpy(values)
print(f"support: {support}")
print(f"values: {values}")

print(n_holes(rle_support))

hole_sizes = rle_support[rle_support[:, 0] > 1, 0] - 1
print(f"hole sizes (k-1): {hole_sizes}")

start_with_hole = rle_support[0, 0] > 1
print(f"start with hole: {start_with_hole}")

seq_sizes = rle_support[rle_support[:, 0] == 1, 0] + 1
if not start_with_hole:
    seq_sizes[0] -= 1
print(f"seq sizes (i+1): {seq_sizes}")

vc_cumsum = np.cumsum(rle_values[:, 1])
print(f"value counts cumsum: {vc_cumsum}")



support: ((1, 3), (3, 1), (1, 3))
values: ((1, 1), (2, 3), (3, 1))
1
hole sizes (k-1): [2]
start with hole: False
seq sizes (i+1): [1 2]
value counts cumsum: [1 4 5]


In [89]:
from typing import Union

# TODO : mettre au point plus tard, il y a des urgences, et cela est du bonus.

def rle_insert_holes(
    rle_support_expr: Union[list, tuple, np.ndarray],
    rle_values_expr: Union[list, tuple, np.ndarray]
) -> tuple:
    """
    Modify a RLE sequence of values expression by inserting "holes" based on support.

    Parameters
    ----------
    rle_support_expr : List[tuple]
        RLE sequence for support.
    rle_values_expr : List[tuple]
        RLE sequence for values.

    Returns
    -------
    tuple
        RLE sequence of values expression with holes.
    """
    # Convert inputs to NumPy arrays if they are not already
    rle_support = rle_expr_to_numpy(rle_support_expr)
    rle_values = rle_expr_to_numpy(rle_values_expr)

    # Ensure that both support and values have the same length
    # if rle_support.shape[0] != rle_values.shape[0]:
    if rle_support_size(rle_support) != rle_support_size(rle_values):
        raise ValueError("Number of repeats mismatch between the support and values RLE sequences.")

    # Count holes
    h = n_holes(rle_support)
    
    # No holes, keep values as is
    if not h:
        return rle_values_expr
    
    
    
    ## La suite est bad..
    
    modified_values.append(values)

    # Create an empty list to store the modified RLE values
    modified_values = []

    for i in range(rle_support.shape[0]):
        support = rle_support[i]
        values = rle_values[i]

        if support[0] == 1:
            # No holes, keep values as is
            modified_values.append(values)
        else:
            # Calculate the number of holes and their positions
            num_holes = support[0] - 1
            hole_positions = np.cumsum([values[1] + num_holes] + list(support[1:]))

            # Split values into segments before and after holes
            segments = np.split(values, hole_positions[:-1])
            modified_values.extend(segments[:-1])

            # Insert NaN holes between segments
            modified_values.extend([(np.nan, num_holes)] * num_holes)

    # Convert the list of modified values to a NumPy array
    modified_values_array = np.concatenate(modified_values)

    return rle_support, modified_values_array

# Example usage:
rle_support_expr = ((1, 2), (3, 1), (1, 3))
rle_values_expr = ((0, 1), (1, 2), (2, 3))
result_support, result_values = rle_insert_holes(rle_support_expr, rle_values_expr)
print(result_support)
print(result_values)


[[1 2]
 [3 1]
 [1 3]]
[ 0.  1.  1.  2. nan  2. nan  2.  2.  3.]
